In [16]:
# -*- coding: utf-8 -*-
# ==============================================================
# CELL 1: DB 설정 및 공통 함수 정의 (이전과 동일)
# ==============================================================
import requests
import json
import psycopg2
import pandas as pd
import time
from datetime import timedelta, date, datetime


# ⚠️ 1. DB 설정 정보 (PostgreSQL 연결) ⚠️
DB_NAME = "laions_db"    
DB_USER = "postgres" 
DB_PASS = "1111" # 여기에 비밀번호 입력
DB_HOST = "localhost"
DB_PORT = "5432"

# 💡 2. KBO API 설정
KBO_API_BASE_URL = "https://www.koreabaseball.com/ws/Schedule.asmx/GetScheduleList"
API_FIXED_PARAMS = {
    'leId': '1', 
    'srIdList': '0,9,6', 
    'seasonId': '2025', 
    'teamId': ''
}

# 💡 3. 팀 이름 <-> API 코드 매핑 테이블
TEAM_MAPPING = {
    'HT': 'KIA', 'SS': '삼성', 'LG': 'LG', 'OB': '두산', 'KT': 'KT', 
    'SK': 'SSG', 'LT': '롯데', 'HH': '한화', 'NC': 'NC', 'WO': '키움',
    'KIA': 'HT', '삼성': 'SS', 'LG': 'LG', '두산': 'OB', 'KT': 'KT', 
    'SSG': 'SK', '롯데': 'LT', '한화': 'HH', 'NC': 'NC', '키움': 'WO' 
}
CODE_TO_KR = {k: v for k, v in TEAM_MAPPING.items() if len(k) == 2}
KR_TO_CODE = {v: k for k, v in TEAM_MAPPING.items() if len(k) != 2} # 임시

# 💡 4. 테이블 구조 정의 (DROP + CREATE)
KBO_RAW_GAMES_TABLE = "kbo_raw_games_raw_json" # 💡 테이블 이름 변경: JSON 데이터 전체를 저장할 공간
CREATE_TABLE_QUERY = f"""
    DROP TABLE IF EXISTS {KBO_RAW_GAMES_TABLE};
    CREATE TABLE {KBO_RAW_GAMES_TABLE} (
        id SERIAL PRIMARY KEY,
        game_month INTEGER NOT NULL,
        raw_json TEXT NOT NULL
    );
"""
# (DB 연결, 생성, 삽입 함수는 이전 Cell 1과 동일하게 유지됩니다.)

def get_db_connection(db_name=DB_NAME):
    try:
        conn = psycopg2.connect(
            dbname=db_name, user=DB_USER, password=DB_PASS, host=DB_HOST, port=DB_PORT, client_encoding='UTF8'
        )
        return conn
    except Exception as e:
        return None

def create_and_init_table(conn):
    try:
        cur = conn.cursor()
        cur.execute(CREATE_TABLE_QUERY)
        conn.commit()
        print(f"✅ 테이블 초기화 및 '{KBO_RAW_GAMES_TABLE}' (JSON 저장용) 재생성 완료.")
    except Exception as e:
        return False
    return True

# 월별 JSON 데이터를 DB에 저장하는 함수
def insert_raw_json_data(conn, game_month, json_data):
    """수집된 전체 JSON 데이터를 DB에 TEXT 형태로 삽입합니다."""
    SQL_INSERT = f"""
        INSERT INTO {KBO_RAW_GAMES_TABLE} (game_month, raw_json) 
        VALUES (%s, %s);
    """
    if not json_data:
        return 0

    cur = conn.cursor()
    # JSON 딕셔너리를 문자열로 변환하여 저장
    cur.execute(SQL_INSERT, (game_month, json.dumps(json_data)))
    conn.commit()
    return 1 # 1개의 월 데이터 삽입 완료

In [17]:
# -*- coding: utf-8 -*-
# ==============================================================
# CELL 2: 월별 JSON 수집 함수 (API 호출 및 JSON 저장)
# ==============================================================

def fetch_json_by_month(game_month):
    """특정 월의 KBO 경기 데이터 JSON 전체를 반환합니다."""
    params = API_FIXED_PARAMS.copy()
    params['gameMonth'] = str(game_month)
    
    try:
        response = requests.get(KBO_API_BASE_URL, params=params)
        response.raise_for_status() 
        # API가 JSON을 반환하므로 .json()을 사용합니다.
        raw_data = response.json() 
        return raw_data
    except Exception as e:
        print(f"❌ {game_month}월 API 호출 실패: {e}")
        return None

In [18]:
# -*- coding: utf-8 -*-
# =============================================================
# CELL 3: DB 초기화 및 '여러 연도' 데이터 수집 파이프라인 (수정본)
# =============================================================
import time

def main_pipeline_multi_year_collection():
    """
    지정된 모든 연도의 KBO 데이터를 수집하여 DB에 누적 저장하는 최종 파이프라인.
    """
    # ----------------------------------------------------------
    # ⚠️ 1. 여기에 수집할 연도를 모두 적어주세요!
    YEARS_TO_SCRAPE = [2022, 2023, 2024, 2025] 
    # ----------------------------------------------------------

    print("🚀 KBO 월별 JSON 데이터 수집 시작...")
    conn = get_db_connection()
    if conn is None:
        print("❌ DB 연결 실패. 파이프라인을 중단합니다.")
        return

    # ✅ 해결책 1: 테이블 초기화는 딱 한 번만 실행합니다.
    # 이 코드는 테이블을 깨끗하게 비우고 새로 만듭니다.
    print("--- 🗑️ 테이블 초기화 중... ---")
    if not create_and_init_table(conn):
        conn.close()
        return

    total_months_collected = 0
    
    # ✅ 해결책 2: 연도별로 반복하는 외부 루프를 추가합니다.
    for year in YEARS_TO_SCRAPE:
        print(f"\n--- ⚾️ {year}년 시즌 데이터 수집 시작 ---")
        
        # API 파라미터에 현재 연도를 동적으로 설정
        API_FIXED_PARAMS['seasonId'] = str(year)
        
        # KBO 정규 시즌은 보통 3월 또는 4월에 시작해서 10월 또는 11월에 끝납니다.
        for month in range(3, 12):
            print(f"  -> {month}월 데이터 수집 중...")
            
            time.sleep(0.5) # 서버에 부담을 주지 않기 위한 지연 시간
            
            try:
                # 데이터 가져오기 (Cell 2의 함수는 수정할 필요 없음)
                json_data = fetch_json_by_month(month)
                
                # 유효한 데이터가 있을 때만 DB에 저장
                if json_data and json_data.get('rows'):
                    # game_month를 '202403'과 같은 형태로 저장
                    game_month_int = int(f"{year}{str(month).zfill(2)}")
                    insert_count = insert_raw_json_data(conn, game_month_int, json_data)
                    total_months_collected += 1
                    print(f"    ✅ {year}년 {month}월 데이터 DB 저장 완료.")
                else:
                    print(f"    ⚠️ {year}년 {month}월 데이터가 없습니다. (시즌 전/후)")
                    
            except Exception as e:
                print(f"    ❌ {year}년 {month}월 수집 중 오류 발생: {e}")
                
    print(f"\n✅ 전체 데이터 수집 완료! 총 {total_months_collected}개월치 데이터를 DB에 적재했습니다.")
    conn.close()

# --- 메인 함수 실행 ---
if __name__ == "__main__":
    main_pipeline_multi_year_collection()

🚀 KBO 월별 JSON 데이터 수집 시작...
--- 🗑️ 테이블 초기화 중... ---
✅ 테이블 초기화 및 'kbo_raw_games_raw_json' (JSON 저장용) 재생성 완료.

--- ⚾️ 2022년 시즌 데이터 수집 시작 ---
  -> 3월 데이터 수집 중...
    ⚠️ 2022년 3월 데이터가 없습니다. (시즌 전/후)
  -> 4월 데이터 수집 중...
    ✅ 2022년 4월 데이터 DB 저장 완료.
  -> 5월 데이터 수집 중...
    ✅ 2022년 5월 데이터 DB 저장 완료.
  -> 6월 데이터 수집 중...
    ✅ 2022년 6월 데이터 DB 저장 완료.
  -> 7월 데이터 수집 중...
    ✅ 2022년 7월 데이터 DB 저장 완료.
  -> 8월 데이터 수집 중...
    ✅ 2022년 8월 데이터 DB 저장 완료.
  -> 9월 데이터 수집 중...
    ✅ 2022년 9월 데이터 DB 저장 완료.
  -> 10월 데이터 수집 중...
    ✅ 2022년 10월 데이터 DB 저장 완료.
  -> 11월 데이터 수집 중...
    ⚠️ 2022년 11월 데이터가 없습니다. (시즌 전/후)

--- ⚾️ 2023년 시즌 데이터 수집 시작 ---
  -> 3월 데이터 수집 중...
    ⚠️ 2023년 3월 데이터가 없습니다. (시즌 전/후)
  -> 4월 데이터 수집 중...
    ✅ 2023년 4월 데이터 DB 저장 완료.
  -> 5월 데이터 수집 중...
    ✅ 2023년 5월 데이터 DB 저장 완료.
  -> 6월 데이터 수집 중...
    ✅ 2023년 6월 데이터 DB 저장 완료.
  -> 7월 데이터 수집 중...
    ✅ 2023년 7월 데이터 DB 저장 완료.
  -> 8월 데이터 수집 중...
    ✅ 2023년 8월 데이터 DB 저장 완료.
  -> 9월 데이터 수집 중...
    ✅ 2023년 9월 데이터 DB 저장 완료.
  -> 10월 데이터 수집 중...
 

In [19]:
import psycopg2
import json # JSON 파싱을 위해 필요합니다.
from datetime import datetime
from bs4 import BeautifulSoup

# 1. DB 연결 정보
DB_CONFIG = {
    "dbname": "laions_db",
    "user": "postgres",
    "password": "1111",  # 실제 비밀번호로 변경해주세요!
    "host": "localhost",
    "port": "5432"
}

# 2. 테이블 이름 정의
SOURCE_TABLE = "kbo_raw_games_raw_json"
DESTINATION_TABLE = "kbo_cleaned_games"

def parse_schedule_json(monthly_json):
    """
    월별 JSON 데이터 전체(HTML 테이블 구조)를 파싱하여
    정제된 경기 데이터 리스트를 반환하는 함수
    """
    cleaned_games_in_month = []
    # 이제 monthly_json은 진짜 딕셔너리이므로 .get()이 안전하게 동작합니다.
    game_rows = monthly_json.get('rows', [])
    
    for row_data in game_rows:
        try:
            cells = row_data.get('row', [])
            if len(cells) < 5: continue

            play_cell_html = None
            relay_cell_html = None
            for cell in cells:
                if cell.get('Class') == 'play':
                    play_cell_html = cell.get('Text')
                elif cell.get('Class') == 'relay':
                    relay_cell_html = cell.get('Text')
            
            if not play_cell_html or not relay_cell_html: continue

            soup_relay = BeautifulSoup(relay_cell_html, 'html.parser')
            link_tag = soup_relay.find('a')
            href = link_tag['href']
            
            game_id = href.split('gameId=')[1].split('&')[0]
            game_date_str = href.split('gameDate=')[1].split('&')[0]
            game_date = datetime.strptime(game_date_str, '%Y%m%d').date()

            soup_play = BeautifulSoup(play_cell_html, 'html.parser')
            spans = soup_play.find_all('span')
            
            away_team_name = spans[0].text
            home_team_name = spans[-1].text
            
            scores = soup_play.find('em').text.split('vs')
            away_score = int(scores[0].strip())
            home_score = int(scores[1].strip())
            
            winning_team = "무승부"
            if away_score > home_score: winning_team = away_team_name
            elif home_score > away_score: winning_team = home_team_name

            cleaned_games_in_month.append({
                "game_id": game_id, "game_date": game_date, "home_team": home_team_name,
                "away_team": away_team_name, "home_score": home_score,
                "away_score": away_score, "winning_team": winning_team
            })
        except Exception:
            continue
            
    return cleaned_games_in_month

def main():
    """메인 실행 함수"""
    conn = None
    all_cleaned_games = []
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        print("✅ PostgreSQL DB에 성공적으로 연결되었습니다.")

        cur.execute(f"""
            DROP TABLE IF EXISTS {DESTINATION_TABLE};
            CREATE TABLE {DESTINATION_TABLE} (
                game_id VARCHAR(20) PRIMARY KEY, game_date DATE NOT NULL,
                home_team VARCHAR(50) NOT NULL, away_team VARCHAR(50) NOT NULL,
                home_score INT, away_score INT, winning_team VARCHAR(50)
            );
        """)
        print(f"✅ '{DESTINATION_TABLE}' 테이블을 성공적으로 생성했습니다.")

        cur.execute(f"SELECT raw_json FROM {SOURCE_TABLE};")
        raw_json_rows = cur.fetchall()
        print(f"✅ '{SOURCE_TABLE}' 테이블에서 {len(raw_json_rows)}개월치 Raw JSON 데이터를 가져왔습니다.")
        
        for row in raw_json_rows:
            # --- 바로 이 부분을 수정했습니다! ---
            # DB에서 가져온 텍스트(str)를 파이썬 딕셔너리(dict)로 변환
            monthly_json = json.loads(row[0])
            # --------------------------------
            
            parsed_games = parse_schedule_json(monthly_json)
            all_cleaned_games.extend(parsed_games)
        
        print(f"✅ 총 {len(all_cleaned_games)} 경기의 데이터 파싱을 완료했습니다.")
        
        insert_query = f"""
            INSERT INTO {DESTINATION_TABLE} (game_id, game_date, home_team, away_team, home_score, away_score, winning_team)
            VALUES (%(game_id)s, %(game_date)s, %(home_team)s, %(away_team)s, %(home_score)s, %(away_score)s, %(winning_team)s)
            ON CONFLICT (game_id) DO NOTHING;
        """
        cur.executemany(insert_query, all_cleaned_games)
        conn.commit()
        print(f"✅ 정제된 데이터 {cur.rowcount}개를 '{DESTINATION_TABLE}' 테이블에 성공적으로 저장했습니다.")

    except psycopg2.Error as e:
        print(f"DB 오류 발생: {e}")
        if conn: conn.rollback()
    finally:
        if conn:
            cur.close()
            conn.close()
            print("🚪 PostgreSQL 연결을 닫았습니다.")

# --- 메인 함수 실행 ---
if __name__ == "__main__":
    main()

✅ PostgreSQL DB에 성공적으로 연결되었습니다.
✅ 'kbo_cleaned_games' 테이블을 성공적으로 생성했습니다.
✅ 'kbo_raw_games_raw_json' 테이블에서 30개월치 Raw JSON 데이터를 가져왔습니다.
✅ 총 2880 경기의 데이터 파싱을 완료했습니다.
✅ 정제된 데이터 2880개를 'kbo_cleaned_games' 테이블에 성공적으로 저장했습니다.
🚪 PostgreSQL 연결을 닫았습니다.
